In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import matplotlib.dates as mdates

In [3]:
file_path = "output.xlsx"

label_columns =  ['label STOCH-R (14)', 'label MACD (12,26,9)', 'label CCI (20)', 'label ADX (14)', 'label STOCH-RL (15,15,1)', 'label RSI (14)']
pred_columns =  ['pred STOCH-R (14)', 'pred MACD (12,26,9)', 'pred CCI (20)', 'pred ADX (14)', 'pred STOCH-RL (15,15,1)', 'pred RSI (14)']


data = pd.ExcelFile(file_path)
sheet_names = data.sheet_names
for sheet_name in sheet_names :
    sheet = data.parse(sheet_name)
    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import warnings

# ⛔ Suppress all warnings
warnings.filterwarnings("ignore")

output_path = "output.xlsx"
input_path = "Base_Test_2500pts v-Louis.xlsx"

indic_columns = ['MACD (12,26,9)', 'STOCH-R (14)', 'STOCH-RL (15,15,1)', 'RSI (14)', 'ADX (14)', 'CCI (20)']
label_columns =  ['label ' + i for i in indic_columns]
pred_columns =  ['pred ' + i for i in indic_columns]
acc_columns =  ['acc ' + i for i in indic_columns]

excel_output = pd.ExcelFile(output_path)
excel_input = pd.ExcelFile(input_path)
sheet_names = excel_output.sheet_names
dfs = []

for sheet_name in sheet_names:
    # Read input (candlestick) and output (prediction) data
    input_df = excel_input.parse(sheet_name)
    input_df['Date'] = pd.to_datetime(input_df['Date'])

    output_df = excel_output.parse(sheet_name)
    output_df['Date'] = pd.to_datetime(output_df['Date'])
    output_df.sort_values('Date', inplace=True)

    join_df = pd.merge(input_df, output_df, on = 'Date')

    dfs.append(join_df[['Unnamed: 0', 'label STOCH-R (14)', 'label MACD (12,26,9)', 'label CCI (20)', 'label ADX (14)', 'label STOCH-RL (15,15,1)', 'label RSI (14)', 'pred STOCH-R (14)', 'pred MACD (12,26,9)', 'pred CCI (20)', 'pred ADX (14)', 'pred STOCH-RL (15,15,1)', 'pred RSI (14)']])

    print(output_df['Date'].isin(input_df['Date']).all())

    
    input_df.set_index('Date', inplace=True)

    # Create subplots: 1 for candlestick + 1 per indicator
    num_indics = len(indic_columns)
    heights = [3] + [1] * num_indics  # Taller for candle, slimmer for indicators
    fig, axes = plt.subplots(num_indics + 1, 1, figsize=(15, 2 * num_indics + 4),
                             sharex=True, gridspec_kw={'height_ratios': heights})

    # Disable mplfinance warning for too much data
    mpf.plot(input_df, type='candle', style='charles', ax=axes[0],
             volume=False, show_nontrading=True, warn_too_much_data=len(input_df)+100)
    axes[0].set_title(f'{sheet_name} - Candlestick Chart')
    axes[0].grid(True)

    for i, indic in enumerate(indic_columns):
        acc_col = 'acc ' + indic
        label_col = 'label ' + indic
        pred_col = 'pred ' + indic

        # Compute accuracy
        output_df[acc_col] = 1 - np.abs(output_df[label_col] - output_df[pred_col])

        # Rolling (moving) average over 30 days
        temp_df = output_df[['Date', acc_col]].copy()
        temp_df.set_index('Date', inplace=True)
        temp_df.sort_index(inplace=True)
        rolling = temp_df[acc_col].rolling('30D').mean().dropna().reset_index()

        ax = axes[i + 1]
        ax.plot(rolling['Date'], rolling[acc_col], label=indic, color='orange', linewidth=1)
        ax.set_ylabel('')
        ax.set_title(f'{indic} - 30-Day Moving Accuracy')
        ax.grid(True)

    # Format x-axis
    axes[-1].set_xlabel('Date')
    axes[-1].xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    fig.autofmt_xdate()

    plt.tight_layout()
    save_path = f"{sheet_name}_prediction_accuracy.png"
    #plt.savefig(save_path, dpi=300)
    plt.close(fig)


True
True
True
True
True
True
True
True


In [46]:
with pd.ExcelWriter('output2.xlsx') as writer:
    for sheet_name, df in zip(sheet_names, dfs):
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [43]:
import pandas as pd
from xlsxwriter.utility import xl_col_to_name

def fix_datetime_columns(df):
    for col in df.columns:
        # Handle timezone-aware datetimes
        if pd.api.types.is_datetime64tz_dtype(df[col]):
            df[col] = df[col].dt.tz_convert(None)
        
        # Normalize all datetimes (set time to 00:00:00)
        if pd.api.types.is_datetime64_dtype(df[col]):
            df[col] = df[col].dt.normalize()
    return df

# Apply fix to all DataFrames
dfs = [fix_datetime_columns(df) for df in dfs]

with pd.ExcelWriter('output2.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df in zip(sheet_names, dfs):
        df.to_excel(writer, sheet_name=sheet_name, index=False)

        workbook  = writer.book
        worksheet = writer.sheets[sheet_name]
        # Set display to only date (no time)
        date_format = workbook.add_format({'num_format': 'yyyy-mm-dd'})

        # Apply format to datetime columns
        for idx, col in enumerate(df.columns):
            if pd.api.types.is_datetime64_dtype(df[col]):
                col_letter = xl_col_to_name(idx)
                worksheet.set_column(f'{col_letter}:{col_letter}', 20, date_format)


In [11]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2501 entries, 2015-02-12 00:00:00+00:00 to 2025-01-22 00:00:00+00:00
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          2501 non-null   int64  
 1   Open                2501 non-null   float64
 2   High                2501 non-null   float64
 3   Low                 2501 non-null   float64
 4   Close               2501 non-null   float64
 5   Volume              2501 non-null   int64  
 6   Lissage SAV-GOL     2463 non-null   float64
 7   MACD (12,26,9)      2463 non-null   float64
 8   STOCH-R (14)        2463 non-null   float64
 9   STOCH-RL (15,15,1)  2463 non-null   float64
 10  RSI (14)            2463 non-null   float64
 11  ADX (14)            2463 non-null   float64
 12  CCI (20)            2463 non-null   float64
dtypes: float64(11), int64(2)
memory usage: 273.5 KB


In [12]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 732 entries, 253 to 655
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   label STOCH-R (14)        732 non-null    float64            
 1   label MACD (12,26,9)      732 non-null    float64            
 2   label CCI (20)            732 non-null    float64            
 3   label ADX (14)            732 non-null    float64            
 4   label STOCH-RL (15,15,1)  732 non-null    float64            
 5   label RSI (14)            732 non-null    float64            
 6   pred STOCH-R (14)         732 non-null    int64              
 7   pred MACD (12,26,9)       732 non-null    int64              
 8   pred CCI (20)             732 non-null    int64              
 9   pred ADX (14)             732 non-null    int64              
 10  pred STOCH-RL (15,15,1)   732 non-null    int64              
 11  pred RSI (14)         

In [30]:
['label STOCH-R (14)', 'label MACD (12,26,9)', 'label CCI (20)', 'label ADX (14)', 'label STOCH-RL (15,15,1)', 'label RSI (14)']
pred_columns =  ['pred STOCH-R (14)', 'pred MACD (12,26,9)', 'pred CCI (20)', 'pred ADX (14)', 'pred STOCH-RL (15,15,1)', 'pred RSI (14)']
join_df[['Unnamed: 0', 'Date', 'label STOCH-R (14)', 'label MACD (12,26,9)', 'label CCI (20)', 'label ADX (14)', 'label STOCH-RL (15,15,1)', 'label RSI (14)', 'pred STOCH-R (14)', 'pred MACD (12,26,9)', 'pred CCI (20)', 'pred ADX (14)', 'pred STOCH-RL (15,15,1)', 'pred RSI (14)']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   Unnamed: 0                732 non-null    int64              
 1   Date                      732 non-null    datetime64[ns, UTC]
 2   label STOCH-R (14)        732 non-null    float64            
 3   label MACD (12,26,9)      732 non-null    float64            
 4   label CCI (20)            732 non-null    float64            
 5   label ADX (14)            732 non-null    float64            
 6   label STOCH-RL (15,15,1)  732 non-null    float64            
 7   label RSI (14)            732 non-null    float64            
 8   pred STOCH-R (14)         732 non-null    int64              
 9   pred MACD (12,26,9)       732 non-null    int64              
 10  pred CCI (20)             732 non-null    int64              
 11  pred ADX (14)      

In [25]:
np.unique_counts(output_df['Date'])

UniqueCountsResult(values=array([Timestamp('2015-08-24 00:00:00+0000', tz='UTC'),
       Timestamp('2015-09-03 00:00:00+0000', tz='UTC'),
       Timestamp('2015-10-21 00:00:00+0000', tz='UTC'),
       Timestamp('2015-11-05 00:00:00+0000', tz='UTC'),
       Timestamp('2015-11-11 00:00:00+0000', tz='UTC'),
       Timestamp('2015-12-03 00:00:00+0000', tz='UTC'),
       Timestamp('2015-12-07 00:00:00+0000', tz='UTC'),
       Timestamp('2015-12-14 00:00:00+0000', tz='UTC'),
       Timestamp('2015-12-29 00:00:00+0000', tz='UTC'),
       Timestamp('2016-01-04 00:00:00+0000', tz='UTC'),
       Timestamp('2016-02-04 00:00:00+0000', tz='UTC'),
       Timestamp('2016-02-23 00:00:00+0000', tz='UTC'),
       Timestamp('2016-02-25 00:00:00+0000', tz='UTC'),
       Timestamp('2016-03-03 00:00:00+0000', tz='UTC'),
       Timestamp('2016-03-09 00:00:00+0000', tz='UTC'),
       Timestamp('2016-03-15 00:00:00+0000', tz='UTC'),
       Timestamp('2016-03-22 00:00:00+0000', tz='UTC'),
       Timestamp('2016

In [ ]:
output_path = 'output2.xlsx'

indic_columns = ['MACD (12,26,9)', 'STOCH-R (14)', 'STOCH-RL (15,15,1)', 'RSI (14)', 'ADX (14)', 'CCI (20)']
label_columns =  ['label ' + i for i in indic_columns]
pred_columns =  ['pred ' + i for i in indic_columns]
acc_columns =  ['acc ' + i for i in indic_columns]

excel_output = pd.ExcelFile(output_path)
excel_input = pd.ExcelFile(input_path)
sheet_names = excel_output.sheet_names
dfs = []

In [5]:
import pandas as pd

# File paths
file1 = "output\\squeezenet_multilabel2.xlsx"
file2 = "Base_Test_2500pts v-Louis.xlsx"
output_file = "merged_common_sheets3.xlsx"

# Load Excel files
xls1 = pd.ExcelFile(file1)
xls2 = pd.ExcelFile(file2)

# Identify common sheet names
common_sheets = set(xls1.sheet_names).intersection(xls2.sheet_names)

# Writer for the output file
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet in common_sheets:
        # Read both sheets
        df1 = pd.read_excel(xls1, sheet_name=sheet)
        df2 = pd.read_excel(xls2, sheet_name=sheet)

        # Join on 'Unnamed: 0'
        if 'Date' in df1.columns and 'Date' in df2.columns:
            merged_df = pd.merge(df1, df2, on='Date', suffixes=('_file1', '_file2'), how= 'outer')
            # Save to new Excel file
            merged_df.to_excel(writer, sheet_name=sheet, index=False)
        else:
            print(f"Skipping sheet '{sheet}' because 'Unnamed: 0' column not found in both files.")


In [ ]:
import pandas as pd

indic = ['label STOCH-R (14)', 'label MACD (12,26,9)', 'label CCI (20)', 'label ADX (14)', 'label STOCH-RL (15,15,1)', 'label RSI (14)', 'pred STOCH-R (14)', 'pred MACD (12,26,9)', 'pred CCI (20)', 'pred ADX (14)', 'pred STOCH-RL (15,15,1)', 'pred RSI (14)']

# File paths
file1 = "output2.xlsx"
file2 = "Base_Test_2500pts v-Louis.xlsx"
output_file = "merged_common_sheets.xlsx"

# Load Excel files
xls1 = pd.ExcelFile(file1)
xls2 = pd.ExcelFile(file2)

# Identify common sheet names
common_sheets = set(xls1.sheet_names).intersection(xls2.sheet_names)

# Writer for the output file
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet in common_sheets:
        # Read both sheets
        df1 = pd.read_excel(xls1, sheet_name=sheet)
        df2 = pd.read_excel(xls2, sheet_name=sheet)

        # Replace 0s with -1 in df1 (output2.xlsx)
        df1[indic] = df1[indic].replace(0, -1)

        # Join on 'Unnamed: 0' (inner join by default)
        if 'Unnamed: 0' in df1.columns and 'Unnamed: 0' in df2.columns:
            merged_df = pd.merge(df1, df2, on='Unnamed: 0', suffixes=('_file1', '_file2'), s = 'outer')
            # Fill empty cells (NaNs) with 0
            merged_df = merged_df.fillna(0)
            # Save to new Excel file
            merged_df.to_excel(writer, sheet_name=sheet, index=False)
        else:
            print(f"Skipping sheet '{sheet}' because 'Unnamed: 0' column not found in both files.")


In [ ]:
indic = ['label STOCH-R (14)', 'label MACD (12,26,9)', 'label CCI (20)', 'label ADX (14)', 'label STOCH-RL (15,15,1)', 'label RSI (14)', 'pred STOCH-R (14)', 'pred MACD (12,26,9)', 'pred CCI (20)', 'pred ADX (14)', 'pred STOCH-RL (15,15,1)', 'pred RSI (14)']

# File paths
file1 = "output2.xlsx"
file2 = "Base_Test_2500pts v-Louis.xlsx"
output_file = "merged_common_sheets.xlsx"


def inference_merge()
    # Load Excel files
    xls1 = pd.ExcelFile(file1)
    xls2 = pd.ExcelFile(file2)

    # Identify common sheet names
    common_sheets = set(xls1.sheet_names).intersection(xls2.sheet_names)

    # Writer for the output file
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for sheet in common_sheets:
            # Read both sheets
            df1 = pd.read_excel(xls1, sheet_name=sheet)
            df2 = pd.read_excel(xls2, sheet_name=sheet)

            # Replace 0s with -1 in df1 (output2.xlsx)
            df1[indic] = df1[indic].replace(0, -1)

            # Join on 'Unnamed: 0' (inner join by default)
            if 'Unnamed: 0' in df1.columns and 'Unnamed: 0' in df2.columns:
                merged_df = pd.merge(df1, df2, on='Unnamed: 0', suffixes=('_file1', '_file2'), s = 'outer')
                # Fill empty cells (NaNs) with 0
                merged_df = merged_df.fillna(0)
                # Save to new Excel file
                merged_df.to_excel(writer, sheet_name=sheet, index=False)
            else:
                print(f"Skipping sheet '{sheet}' because 'Unnamed: 0' column not found in both files.")




In [ ]:
from inference import format_datetime_column, inference, combine_arrays_to_df

def inference_merged_df(model, test_loader, indics, raw_data_path, DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu"), output_path= None) :
    label_columns =  ['label ' + i for i in indics]
    pred_columns =  ['pred ' + i for i in indics]
    all_labels, all_preds, _, all_dates,  all_sheet = inference (model, test_loader, indics, DEVICE)

    arrays = [all_dates, all_labels, all_preds, all_sheet]
    columns = ['Date'] + label_columns + pred_columns  + ['Sheet']

    df = combine_arrays_to_df(arrays, columns)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    df_xlsx = format_datetime_column(df, 'Date')

    raw_xlsx = pd.ExcelFile(raw_data_path)
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for sheet_name, group in df_xlsx.groupby('Sheet'):
            # Clean the sheet name to be Excel-safe (max 31 chars, no special chars)
            safe_sheet_name = str(sheet_name)[:31].replace('/', '_')
            df_1 = group.drop(columns='Sheet')
            df_2 = pd.read_excel(df_xlsx, df_xlsx)
            merged_df = pd.merge(df_1, df_2, on='Date', suffixes=('_file1', '_file2'), s = 'outer')
            merged_df.to_excel(writer, sheet_name=sheet, index=False)

    print(f"✅ Excel file '{output_path}' saved with multiple sheets.")

    return merged_df

df = inference_merged_df()

In [1]:
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from models import load_model_squeezenet
from dataset import get_dataloader
import os
from inference import format_datetime_column

output_path = "output\\merged\\squeezenet_multilabel_synth.xlsx"

temp_xlsx = pd.ExcelFile('output\\squeezenet_multilabel_synth.xlsx')

raw_xlsx = pd.ExcelFile('Base_Test_2500pts avec Synthétiques.xlsx')

print(raw_xlsx.sheet_names)
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for sheet_name in temp_xlsx.sheet_names :
        # Clean the sheet name to be Excel-safe (max 31 chars, no special chars)
        safe_sheet_name = str(sheet_name)[:31].replace('/', '_')
        print(safe_sheet_name)
        df_1 = pd.read_excel(temp_xlsx, safe_sheet_name)
        df_1['Date'] = pd.to_datetime(df_1['Date'], utc=True)
        df_1 = df_1.sort_values('Date').reset_index(drop=True)
        df_1 = format_datetime_column(df_1, 'Date')
        df_2 = pd.read_excel(raw_xlsx, safe_sheet_name)

        print(df_1.columns)
        print(df_2.columns)
        if 'Date' in df_1.columns and 'Date' in df_2.columns:
            merged_df = pd.merge(df_1, df_2, on='Date', suffixes=('_file1', '_file2'), how='outer')
            if not merged_df.empty:
                merged_df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            else:
                print(f"⚠️ Merged DataFrame for {safe_sheet_name} is empty.")
        else:
            print(f"❌ Skipping sheet '{safe_sheet_name}' due to missing 'Date' column.")

print(f"✅ Excel file '{output_path}' saved with multiple sheets.")

['EURUSDm1', 'EURUSDm5_p1', 'EURUSDm5_p2', 'EURUSDh1_p1', 'EURUSDh1_p2', 'CAC-40h4_p1', 'CAC-40h4_p2', 'CAC-40d1_p1', 'CAC-40d1_p2', 'Ss1', 'Ss2', 'Ss3', 'Ss4', 'Ss5']
CAC-40d1_p2
Index(['Date', 'label MACD (12,26,9)', 'label STOCH-R (14)',
       'label STOCH-RL (15,15,1)', 'label RSI (14)', 'label ADX (14)',
       'label CCI (20)', 'pred MACD (12,26,9)', 'pred STOCH-R (14)',
       'pred STOCH-RL (15,15,1)', 'pred RSI (14)', 'pred ADX (14)',
       'pred CCI (20)'],
      dtype='object')
Index(['Unnamed: 0', 'Date', 'Open', 'High', 'Low', 'Close', 'Lissage SAV-GOL',
       'MACD (12,26,9)', 'STOCH-R (14)', 'STOCH-RL (15,15,1)', 'RSI (14)',
       'ADX (14)', 'CCI (20)'],
      dtype='object')


IndexError: At least one sheet must be visible

In [2]:
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from models import load_model_squeezenet
from dataset import get_dataloader
import os
from inference import format_datetime_column

# Define file paths
output_path = "output\\merged\\squeezenet_multilabel_synth.xlsx"
temp_xlsx = pd.ExcelFile("output\\squeezenet_multilabel_synth.xlsx")
raw_xlsx = pd.ExcelFile("Base_Test_2500pts avec Synthétiques.xlsx")

print("📄 Sheets in raw file:", raw_xlsx.sheet_names)

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for sheet_name in temp_xlsx.sheet_names:
        # Make sheet name Excel-safe
        safe_sheet_name = str(sheet_name)[:31].replace("/", "_")
        print(f"\n🔄 Processing sheet: {safe_sheet_name}")

        # Load predicted (temp) and raw data
        df_1 = pd.read_excel(temp_xlsx, safe_sheet_name)
        df_2 = pd.read_excel(raw_xlsx, safe_sheet_name)

        # Parse and format 'Date' column
        if 'Date' in df_1.columns:
            df_1['Date'] = pd.to_datetime(df_1['Date'], utc=True)
            df_1 = df_1.sort_values('Date').reset_index(drop=True)
            df_1 = format_datetime_column(df_1, 'Date')
        else:
            print(f"❌ 'Date' column missing in predictions ({safe_sheet_name}), skipping.")
            continue

        if 'Date' not in df_2.columns:
            print(f"❌ 'Date' column missing in raw data ({safe_sheet_name}), skipping.")
            continue

        # Display debug info
        print("📊 Columns in prediction:", df_1.columns.tolist())
        print("📊 Columns in raw:", df_2.columns.tolist())

        # Merge both on 'Date'
        merged_df = pd.merge(df_1, df_2, on="Date", suffixes=('_file1', '_file2'), how="outer")

        if not merged_df.empty:
            merged_df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            print(f"✅ Sheet '{safe_sheet_name}' written successfully.")
        else:
            print(f"⚠️ Merged DataFrame for '{safe_sheet_name}' is empty — skipped writing.")

print(f"\n✅ Excel file '{output_path}' saved with multiple sheets.")


📄 Sheets in raw file: ['EURUSDm1', 'EURUSDm5_p1', 'EURUSDm5_p2', 'EURUSDh1_p1', 'EURUSDh1_p2', 'CAC-40h4_p1', 'CAC-40h4_p2', 'CAC-40d1_p1', 'CAC-40d1_p2', 'Ss1', 'Ss2', 'Ss3', 'Ss4', 'Ss5']

🔄 Processing sheet: CAC-40d1_p2
📊 Columns in prediction: ['Date', 'label MACD (12,26,9)', 'label STOCH-R (14)', 'label STOCH-RL (15,15,1)', 'label RSI (14)', 'label ADX (14)', 'label CCI (20)', 'pred MACD (12,26,9)', 'pred STOCH-R (14)', 'pred STOCH-RL (15,15,1)', 'pred RSI (14)', 'pred ADX (14)', 'pred CCI (20)']
📊 Columns in raw: ['Unnamed: 0', 'Date', 'Open', 'High', 'Low', 'Close', 'Lissage SAV-GOL', 'MACD (12,26,9)', 'STOCH-R (14)', 'STOCH-RL (15,15,1)', 'RSI (14)', 'ADX (14)', 'CCI (20)']


IndexError: At least one sheet must be visible

In [8]:
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from models import load_model_squeezenet
from dataset import get_dataloader
import os
from inference import format_datetime_column

# Define paths
output_path = "output\\merged\\squeezenet_multilabel_synth.xlsx"
temp_xlsx = pd.ExcelFile("output\\squeezenet_multilabel_synth.xlsx")
raw_xlsx = pd.ExcelFile("Base_Test_2500pts avec Synthétiques.xlsx")

print("📄 Sheets found in raw file:", raw_xlsx.sheet_names)

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    successful_sheets = 0

    for sheet_name in temp_xlsx.sheet_names:
        safe_sheet_name = str(sheet_name)[:31].replace("/", "_")
        print(f"\n🔄 Processing sheet: {safe_sheet_name}")

        try:
            # Read both sheets
            df_1 = pd.read_excel(temp_xlsx, safe_sheet_name)
            df_2 = pd.read_excel(raw_xlsx, safe_sheet_name)
        except Exception as e:
            print(f"❌ Failed to read sheet '{safe_sheet_name}': {e}")
            continue

        # Ensure both have 'Date'
        if 'Date' not in df_1.columns or 'Date' not in df_2.columns:
            df_2['Date'] = pd.date_range(start='2023-01-01', periods=len(df_2), freq='D')

        try:
            # Convert to UTC and format
            df_1['Date'] = pd.to_datetime(df_1['Date'], utc=True)
            df_2['Date'] = pd.to_datetime(df_2['Date'], utc=True)

            df_1 = df_1.sort_values('Date').reset_index(drop=True)
            df_1 = format_datetime_column(df_1, 'Date')
            df_2 = format_datetime_column(df_2, 'Date')

            # Merge on 'Date'
            merged_df = pd.merge(df_1, df_2, on='Date', suffixes=('_file1', '_file2'), how='outer')

            if not merged_df.empty:
                merged_df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                successful_sheets += 1
                print(f"✅ Sheet '{safe_sheet_name}' written.")
            else:
                print(f"⚠️ Empty merged DataFrame for '{safe_sheet_name}', skipped.")

        except Exception as e:
            print(f"❌ Error processing sheet '{safe_sheet_name}': {e}")

    # Fallback: write dummy sheet if no sheet succeeded
    if successful_sheets == 0:
        pd.DataFrame({'Info': ['No valid data or merges completed.']}).to_excel(writer, sheet_name="Info")
        print("⚠️ No sheets were successfully written. Dummy sheet 'Info' added.")

print(f"\n✅ Excel file saved: '{output_path}'")


📄 Sheets found in raw file: ['EURUSDm1', 'EURUSDm5_p1', 'EURUSDm5_p2', 'EURUSDh1_p1', 'EURUSDh1_p2', 'CAC-40h4_p1', 'CAC-40h4_p2', 'CAC-40d1_p1', 'CAC-40d1_p2', 'Ss1', 'Ss2', 'Ss3', 'Ss4', 'Ss5']

🔄 Processing sheet: CAC-40d1_p2
✅ Sheet 'CAC-40d1_p2' written.

🔄 Processing sheet: CAC-40h4_p2
✅ Sheet 'CAC-40h4_p2' written.

🔄 Processing sheet: EURUSDh1_p2


Exception ignored in: <function ZipFile.__del__ at 0x00000195314D2C00>
Traceback (most recent call last):
  File "C:\Users\louis\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1980, in __del__
    self.close()
  File "C:\Users\louis\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1997, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


✅ Sheet 'EURUSDh1_p2' written.

🔄 Processing sheet: EURUSDm5_p2
✅ Sheet 'EURUSDm5_p2' written.

🔄 Processing sheet: Ss1
✅ Sheet 'Ss1' written.

🔄 Processing sheet: Ss2
✅ Sheet 'Ss2' written.

🔄 Processing sheet: Ss3
✅ Sheet 'Ss3' written.

🔄 Processing sheet: Ss4
✅ Sheet 'Ss4' written.

🔄 Processing sheet: Ss5
✅ Sheet 'Ss5' written.

✅ Excel file saved: 'output\merged\squeezenet_multilabel_synth.xlsx'
